---
# Tạo collection trên chromaDB

In [34]:
from google.genai import types
import pandas as pd
from google import genai
import chromadb
from chromadb.config import Settings
import os
import sys
import numpy as np
from dotenv import load_dotenv, find_dotenv
from numpy.linalg import norm
from typing import List
import time

In [11]:
pip show chromadb

Python(31416) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Name: chromadb
Version: 1.0.20
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.11/site-packages
Requires: bcrypt, build, grpcio, httpx, importlib-resources, jsonschema, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-sdk, orjson, overrides, posthog, pybase64, pydantic, pypika, pyyaml, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


---
# embedding + push

In [ ]:
# Load environment variables from .env file
load_dotenv('/Users/macbook/Desktop/BangA_DSC2025/.env')

# --- 1. Cấu hình ChromaDB và Gemini API ---
try:
    # Lấy ChromaDB Host và Port từ biến môi trường
    # Hoặc sử dụng giá trị mặc định nếu không có
    CHROMA_SERVER_HOST = os.environ.get("CHROMA_SERVER_HOST")
    CHROMA_SERVER_PORT = int(os.environ.get("CHROMA_SERVER_PORT", 8000))
    
    # Kết nối đến máy chủ ChromaDB trên VPS
    client_chroma = chromadb.HttpClient(
        host=CHROMA_SERVER_HOST,
        port=CHROMA_SERVER_PORT
    )

    # In ra thông tin kết nối để kiểm tra
    print(f"✅ Đã kết nối thành công tới ChromaDB server tại: {CHROMA_SERVER_HOST}:{CHROMA_SERVER_PORT}")
    print(f"Số collection hiện có: {client_chroma.count_collections()}")

    # Lấy một danh sách tất cả các Gemini API Key từ biến môi trường
    gemini_api_keys: List[str] = [
        os.environ[key]
        for key in os.environ
        if key.startswith("API_GEMINI_")
    ]
    gemini_api_keys = [os.getenv(f"API_GEMINI_{i}") for i in range(0, 12)]
    gemini_api_keys = [key for key in gemini_api_keys if key]  # Lọc key hợp lệ
    if not gemini_api_keys:
        raise ValueError("Không tìm thấy Gemini API keys nào bắt đầu bằng 'API_GEMINI_' trong biến môi trường.")
    
    print(f"✅ Đã cấu hình {len(gemini_api_keys)} Gemini API keys.")

except Exception as e:
    print(f"❌ Lỗi cấu hình hoặc kết nối: {e}", file=sys.stderr)
    sys.exit(1)


In [ ]:
# nếu chưa có collection thì tạo lại
collection_respiratory = client_chroma.create_collection(name="respiratory", embedding_function=None)
collection_allergy = client_chroma.create_collection(name="allergy", embedding_function=None)
collection_heat = client_chroma.create_collection(name="heat", embedding_function=None)
collection_cardiovascular = client_chroma.create_collection(name="cardiovascular", embedding_function=None)

print("Collections created successfully:")
print(collection_respiratory.name, collection_allergy.name, collection_heat.name, collection_cardiovascular.name)


In [54]:
# --- 2. Chuẩn bị dữ liệu từ DataFrame (ví dụ) ---

doc_chunks = pd.read_csv('corpus/respiratory_chunks.csv')


print(f"\n📋 Đã load {len(doc_chunks)} chunks từ DataFrame.")
print("Dữ liệu ví dụ:")
print(doc_chunks)


📋 Đã load 274 chunks từ DataFrame.
Dữ liệu ví dụ:
                                                   url  \
0    https://www.sriramakrishnahospital.com/blog/pu...   
1    https://www.sriramakrishnahospital.com/blog/pu...   
2    https://www.sriramakrishnahospital.com/blog/pu...   
3    https://www.sriramakrishnahospital.com/blog/pu...   
4    https://www.sriramakrishnahospital.com/blog/pu...   
..                                                 ...   
269  https://www.epa.gov/pmcourse/patient-exposure-...   
270  https://www.weather.gov/safety/airquality?utm_...   
271  https://www.weather.gov/safety/airquality?utm_...   
272  https://www.weather.gov/safety/airquality?utm_...   
273  https://www.weather.gov/safety/airquality?utm_...   

                                                 chunk  
0    Usually, respiratory disease will affect most ...  
1    Further, this blog will unclose how summer hea...  
2    Further, let’s see how summer heat affects res...  
3    Automatically you w

In [55]:
# --- 3. Chọn và kết nối đến Collection ---
# lựa chọn collection
COLLECTION_NAME = "respiratory"
try:
    collection = client_chroma.get_or_create_collection(name=COLLECTION_NAME, embedding_function=None)
    print(f"\n🎯 Đã kết nối tới collection: '{collection.name}'")
except Exception as e:
    print(f"❌ Lỗi khi kết nối tới collection '{COLLECTION_NAME}': {e}", file=sys.stderr)
    sys.exit(1)


🎯 Đã kết nối tới collection: 'respiratory'


In [56]:
print(collection)

Collection(name=respiratory)


In [ ]:
# --- 4. Thực hiện Embedding và Add vào Collection ---
print("\n🚀 Bắt đầu tạo embedding và tải dữ liệu lên ChromaDB...")

# Tạo danh sách để lưu trữ dữ liệu hàng loạt
documents_to_add = []
embeddings_to_add = []
metadatas_to_add = []
ids_to_add = []

api_key_index = 0
chunk_index = 0
while chunk_index < len(doc_chunks):
    # Cấu hình Gemini API với key hiện tại trong danh sách
    current_api_key = gemini_api_keys[api_key_index % len(gemini_api_keys)]
    print('đang dùng api_key: ', current_api_key)

    client_genai = genai.Client(api_key=current_api_key)
    row = doc_chunks.iloc[chunk_index]
    text_chunk = row['chunk']
    source_url = row['url']
    doc_id = f"doc_{chunk_index}"

    try:
        print(f"Đang xử lý chunk {chunk_index+1}/{len(doc_chunks)} với API Key thứ {api_key_index % len(gemini_api_keys) + 1}...")
        
        # Tạo embedding với Gemini API
        result = client_genai.models.embed_content(
            model="gemini-embedding-001",
            contents=text_chunk,
            config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT", output_dimensionality=3072)
        )

        # Trích xuất và chuẩn hoá embedding
        [embedding_obj] = result.embeddings
        embedding_values_np = np.array(embedding_obj.values)
        normed_embedding = embedding_values_np / np.linalg.norm(embedding_values_np)
        
        # Thêm vào danh sách hàng loạt
        documents_to_add.append(text_chunk)
        embeddings_to_add.append(normed_embedding.tolist())
        metadatas_to_add.append({"source": source_url})
        ids_to_add.append(doc_id)
        
        # Tăng chỉ số chunk và tạm dừng để tránh rate limit (RPM)
        chunk_index += 1
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi khi xử lý chunk '{doc_id}': {e}", file=sys.stderr)
        if "quota" in str(e).lower() or "rate limit" in str(e).lower():
            # Nếu gặp lỗi giới hạn tốc độ, chuyển sang API key tiếp theo
            print(f"Giới hạn tốc độ đã đạt. Chuyển sang API Key kế tiếp...")
            api_key_index += 1
            if api_key_index >= len(gemini_api_keys):
                print("Hết tất cả các API keys. Vui lòng thử lại sau.")
                break # Dừng vòng lặp nếu hết keys
        else:
            # Nếu là lỗi khác, bỏ qua chunk và tiếp tục
            print("Gặp lỗi không liên quan đến giới hạn tốc độ. Bỏ qua chunk này.")
            chunk_index += 1
            
# Thêm tất cả dữ liệu vào collection trong một lần
if documents_to_add:
    try:
        collection.add(
            documents=documents_to_add,
            embeddings=embeddings_to_add,
            metadatas=metadatas_to_add,
            ids=ids_to_add
        )
        print(f"\n✅ Đã tải lên {len(documents_to_add)} chunks thành công!")
    except Exception as e:
        print(f"❌ Lỗi khi tải dữ liệu lên ChromaDB: {e}", file=sys.stderr)
else:
    print("\n⚠️ Không có chunk nào được xử lý thành công để tải lên.")

print("\n🎉 Hoàn tất quá trình.")

In [31]:
count = collection.count()
print(f"✅ Collection có tổng cộng {count} records.")

✅ Collection có tổng cộng 274 records.


In [32]:
top_records = collection.peek(limit=5)

In [ ]:
print("\n🔍 Top 5 records trong collection:")
for doc, metadata, doc_id, embed in zip(top_records['documents'], top_records['metadatas'], top_records['ids'], top_records['embeddings']):
    print(f"---")
    print(f"ID: {doc_id}")
    print(f"Metadata: {metadata}")
    print(f"Content: {doc[:50]}...") # Print a truncated version of the document
    print(f"embeddings: {embed}")

---

In [6]:
text = doc_chunks.at[0,'chunk']
print(text)

Usually, respiratory disease will affect most of us during winter due to the cold air, flu season, and more indoor pollution. Have ever imagined that summer heat could take a toll on your respiratory health? For individuals with asthma, chronic obstructive pulmonary disease (COPD), bronchitis, or other lung conditions, the hot and humid summer air can act as a silent trigger, this will worsen the situation and even breathing becomes harder. Further, this blog will unclose how summer heat disturbs the respiratory system and what you can do to protect yourself. Why is summer heat a problem for respiratory health?


In [ ]:
client_test = genai.Client(api_key="YOUR_API_KEY")

result = client_test.models.embed_content(
    model="gemini-embedding-001",
    contents=text,
    config=types.EmbedContentConfig(output_dimensionality=768)
)

[embedding_obj] = result.embeddings
embedding_length = len(embedding_obj.values)

print(f"Length of embedding: {embedding_length}")

Length of embedding: 768
